# Collaboration and Competition - Tennis

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

import torch
import matplotlib.pyplot as plt
%matplotlib inline
from collections import deque
import time


In [4]:
env = UnityEnvironment(file_name="Tennis.app")


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [6]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [7]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.09000000171363354
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.09000000171363354


When finished, you can close the environment.

In [8]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [8]:
from ddpg_agent import Agent

random_seed = 7
train_mode = True

agent = Agent(state_size=state_size, action_size=action_size, random_seed=2)
n_episodes = 1000
max_t = 1000
print_every = 5
learn_every = 20
goal_score = 0.5
num_learn = 10

In [9]:
def ddpg(n_episodes=n_episodes, max_t=max_t, print_every=print_every, learn_every=learn_every, 
         num_learn=num_learn, goal_score=goal_score):
    total_scores_deque = deque(maxlen=100)
    total_scores = []
    total_average_score = []
    average_global = []
    
    for i_episode in range(1, n_episodes+1):
        #print("episode", i_episode)
        # Reset Env and Agent
        env_info = env.reset(train_mode=train_mode)[brain_name]       # reset the environment    
        states = env_info.vector_observations                   # get the current state (for each agent)
        scores = np.zeros(num_agents)                            # initialize the score (for each agent)
        agent.reset()
        
        start_time = time.time()
        
        for t in range(max_t):
            actions = agent.act(states)
            
            env_info = env.step(actions)[brain_name]            # send all actions to the environment
            next_states = env_info.vector_observations          # get next state (for each agent)
            rewards = env_info.rewards                          # get reward (for each agent)
            dones = env_info.local_done                         # see if episode finished
            
            agent.step(states, actions, rewards, next_states, dones, t) # send actions to the agent
            
            scores += env_info.rewards                           # update the score (for each agent)
            states = next_states                                # roll over states to next time step
            
            
            if np.any(dones):                                   # exit loop if episode finished
                break
    
        mean_score = np.mean(scores)
        min_score = np.min(scores)
        max_score = np.max(scores)
        total_scores_deque.append(mean_score)
        total_scores.append(mean_score)
        total_average_score = np.mean(total_scores_deque)
        average_global.append(total_average_score)
        duration = time.time() - start_time
        
        
        if i_episode % print_every == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            #print('\rEpisode {}\tTotal Average Score: {:.2f}'.format(i_episode, total_average_score))  
            print('\rEpisode {}\tTotal Average Score: {:.2f}\tMean: {:.2f}\tMin: {:.2f}\tMax: {:.2f}\tDuration: {:.2f}'
              .format(i_episode, total_average_score, mean_score, min_score, max_score, duration))
            
        if total_average_score >= goal_score and i_episode >= 100:
            print('Problem Solved after {} epsisodes!! Total Average score: {:.2f}'.format(i_episode, total_average_score))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break
    
    return total_scores, average_global

total_scores, average_global = ddpg()


Episode 5	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.04
Episode 10	Total Average Score: 0.00	Mean: 0.05	Min: 0.00	Max: 0.09	Duration: 0.43
Episode 15	Total Average Score: 0.00	Mean: 0.05	Min: -0.01	Max: 0.10	Duration: 0.36
Episode 20	Total Average Score: 0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.18
Episode 25	Total Average Score: 0.00	Mean: 0.05	Min: 0.00	Max: 0.09	Duration: 0.36
Episode 30	Total Average Score: 0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.34
Episode 35	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.17
Episode 40	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.18
Episode 45	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.19
Episode 50	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.19
Episode 55	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.19
Episode 60	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max:

Episode 485	Total Average Score: 0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.20
Episode 490	Total Average Score: 0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.20
Episode 495	Total Average Score: 0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.23
Episode 500	Total Average Score: 0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.21
Episode 505	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.19
Episode 510	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.19
Episode 515	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.20
Episode 520	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.19
Episode 525	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.20
Episode 530	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.19
Episode 535	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.18
Episode 540	Total Average Score: -0.00	Mean: -0

Episode 960	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.20
Episode 965	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.20
Episode 970	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.19
Episode 975	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.23
Episode 980	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.19
Episode 985	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.23
Episode 990	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.22
Episode 995	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.19
Episode 1000	Total Average Score: -0.00	Mean: -0.00	Min: -0.01	Max: 0.00	Duration: 0.20
